In [2]:
from grizly_dev.grizly import Store, QFrame

QFrame().build_subquery("store.json", subquery="new_5", database="denodo")
#QFrame().build_field("store.json")

ValueError: Field 'material_number' doesn't have custom_type and has invalid value in type: ''. Valid values: 'dim', 'num'.

In [29]:
import ipywidgets as w
import os

current_folder = os.path.abspath("")

options = os.listdir(current_folder)

class PathUI():

    def __init__(self):
        self.current_folder = os.path.abspath("")
        self.options = os.listdir(self.current_folder)
        self.dd = w.Dropdown(options=self.options)
        self.dds = [self.dd]
        self.label = w.Label(value=self.current_folder)
        self.button = w.Button(description="+")
        self.button_minus = w.Button(description="-")
        buttons = w.HBox([self.button, self.button_minus])
        self.out = w.VBox([self.label, buttons, self.dd])
        self.button.on_click(self._btn_update)
        
    def _btn_update(self, button):
        if len(self.dds) > 1:
            paths = []
            for _dd in self.dds:
                paths.append(_dd.value)
            curr_folder = os.path.join(os.path.abspath(""), *paths)
            self.label.value = str(curr_folder)
        else:
            curr_folder = os.path.abspath(self.dd.value)
            self.label.value = str(curr_folder)
            
        try:
            options = os.listdir(curr_folder)
            dd = w.Dropdown(options=options)
            self.dds.append(dd)
            self.out.children += (dd, )
        except NotADirectoryError:
            pass
    
    def build_path(self):
        return self.out
    
PathUI().build_path()

In [16]:
Config.paths.

ModuleNotFoundError: No module named 'get_cells'

In [24]:
from grizly import Config
d = standard = {
        "config": {
            "standard": {
            "email": {
                "email_address": "my_email@example.com",
                "email_password": "my_password",
                "send_as": "Team"
            }}}}
Config().from_dict(d)

In[23]

Config data has been saved.


'from grizly import Config\nd = standard = {\n        "config": {\n            "standard": {\n            "email": {\n                "email_address": "my_email@example.com",\n                "email_password": "my_password",\n                "send_as": "Team"\n            }}}}\nConfig().from_dict(d)\n\nIn'

In [27]:
In[23].replace("from", "test")

'test grizly import Config\nd = standard = {\n        "config": {\n            "standard": {\n            "email": {\n                "email_address": "my_email@example.com",\n                "email_password": "my_password",\n                "send_as": "Team"\n            }}}}\nConfig().test_dict(d)\n\nIn'

In [28]:
from IPython.core.display import HTML

h = HTML("""<input type="file">""")
h

In [1]:
import ipywidgets as w
label = w.HTML("<h2 style='text-align:center'>Test</h2>")
schema = w.Text(value='base_views', description='Schema:', disabled=False, layout=w.Layout(height='auto', width='auto'))
table = w.Text(value='sales_monthly_close', description='Table:', disabled=False, layout=w.Layout(height='auto', width='auto'))
engine = w.Text(value='mssql+pyodbc://acoe_redshift', description='Engine:', disabled=False, layout=w.Layout(height='auto', width='auto'))
button = w.Button(description='Get Columns',disabled=False, layout=w.Layout(height='auto', width='auto'))

app = w.AppLayout(header=label,
          left_sidebar=w.VBox([schema, table, engine, button]),
          center=None,
          right_sidebar=None,
          footer=None)
app

AppLayout(children=(HTML(value="<h2 style='text-align:center'>Test</h2>", layout=Layout(grid_area='header')), …

In [3]:
cols = ["abc", "adc", "aaa", "bbb"]
checks = w.Box([w.Checkbox(description=col, layout=w.Layout(height='auto', width='auto')) for col in cols]
               , layout = w.Layout(height='auto', width='auto', column='center'))
app.center = checks

In [140]:
import ipywidgets as w
from grizly_dev.grizly import Store, QFrame

class UIField():
    
    def __init__(self, store_path):
        self.store_path = store_path
        self.label = w.Label("Note you need to reload cell for each new field")
        self.field_name = w.Text(value='field_name', description='Field Name:', disabled=False)
        self.expression = w.Textarea(value='SQL Expression Here'
                    , description='Expression:', disabled=False, layout={'width': '100%'})
        self.subquery = w.Text(value='subquery_name', description='Subquery Name:', disabled=False)
        self.type = w.Text(value='dim', description='Type:', disabled=False)
        self._as = w.Text(value='', description='As:', disabled=False)
        self.group_by = w.Text(value='group', description='Group By:', disabled=False)
        self.fields =  w.Dropdown(options=[])
        self.btn_add_to_store = w.Button(description='Add To Store',disabled=False)
        self.btn_update_store = w.Button(description='Update Store',disabled=False)
        self.btn_add_field = w.Button(description='Add Field',disabled=False)
        self.btn_edit_field = w.Button(description='Edit Field',disabled=False)
        self.output = None
        
    def _generate_qf(self):
        qf = QFrame().read_json(self.store_path, subquery=self.subquery.value)
        return qf
        
    def _save_qf(self, qf):
        qf.save_json(self.store_path, subquery=self.subquery.value)
        
    def _btn_add_field(self, button):
        self.label.value = "You are now adding a new field"
        new_out = self.add_field()
        self.output.children += (new_out,)
        
    def _btn_edit_field(self, button):
        self.label.value = "You are now editing"
        self.btn_add_to_store.display = "none"
        new_out = self.edit_field()
        self.output.children += (new_out,)
    
    def _btn_update_store(self, button):
        qf = self._generate_qf()
        try:
            data = qf.data["select"]["fields"][self.field_name.value]
            data["expression"] = self.expression.value
            data["as"] = self.expression._as
            data["type"] = self.expression.type
            data["group_by"] = self.expression.group_by
            data["expression"] = self.expression.value
            self._save_qf(qf)
        except:
            self.label.value = "Error. Are you sure this fields exists?"
        
    def _btn_add_to_store(self, button):
        qf = self._generate_qf()
        qf.assign(**{self.field_name.value:self.expression.value})
        self._save_qf(qf)
    
    def build_field(self):
        row_1 =  w.HBox([self.label])
        row_2 =  w.HBox([self.subquery])
        row_3 =  w.HBox([self.btn_add_field, self.btn_edit_field])
        first_out = w.VBox([row_1, row_2, row_3])
        self.output = first_out
        self.btn_add_field.on_click(self._btn_add_field)
        self.btn_edit_field.on_click(self._btn_edit_field)
        return self.output
        
    def add_field(self):
        row_1 = w.VBox([self.field_name, self.type, self._as, self.group_by])
        row_2 = w.HBox([self.expression])
        row_3 = w.HBox([self.btn_add_to_store])
        self.btn_add_to_store.on_click(self._btn_add_to_store)
        return w.VBox([row_1, row_2, row_3])
    
    def edit_ui(self):
        qf = self._generate_qf()
        field = qf.data["select"]["fields"][self.fields.value]
        self.expression._as = field["as"]
        self.expression.type = field["type"]
        self.expression.group_by = field["group_by"]
        self.expression.value = field["expression"]
    
    def edit_field(self):
        qf = self._generate_qf()
        fields = qf.get_fields()
        self.fields.options = fields
        self.fields.on_trait_change(self.edit_ui, name="value")
        row_1 = w.VBox([self.fields, self.type, self._as, self.group_by])
        row_2 = w.HBox([self.expression])
        row_3 = w.HBox([self.btn_update_store])
        self.edit_ui()
        self.btn_update_store.on_click(self._btn_update_store)
        return w.VBox([row_1, row_2, row_3])
    
ui = UIField("store.json").build_field()
ui

store.json


In [48]:
qf.data["select"]["fields"]["account_manager"]["expression"] = "test"

In [49]:
qf.data

{'select': {'table': 'sales_monthly_close',
  'schema': 'base_views',
  'fields': {'account_manager': {'type': 'dim',
    'as': '',
    'group_by': '',
    'order_by': '',
    'expression': 'test',
    'select': '',
    'custom_type': ''},
   'field': {'type': 'dim',
    'as': 'field',
    'group_by': '',
    'order_by': '',
    'expression': 'some',
    'custom_type': ''},
   'field_name': {'type': 'dim',
    'as': 'field_name',
    'group_by': '',
    'order_by': '',
    'expression': 'some',
    'custom_type': ''},
   'field_nameww': {'type': 'dim',
    'as': 'field_nameww',
    'group_by': '',
    'order_by': '',
    'expression': "case\n\n'fiscal_year' = 2014 then 1\nelse 0\n\nend",
    'custom_type': ''}},
  'engine': 'mssql+pyodbc://redshift_acoe',
  'where': '',
  'distinct': '',
  'having': '',
  'limit': ''}}